<a href="https://colab.research.google.com/github/mish841/RAISE-26-Data-Science-AI-Competition-Submission/blob/mariam-phase2-progress/Data_Science_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
from collections import Counter

DATA_PATH = "dataset_A_news_full_10500.csv"
# df = pd.read_csv("dataset_A_news_full_10500.csv")


df = pd.read_csv(DATA_PATH)

df.head()
df.info()
df.isna().sum().sort_values(ascending=False).head(20)
# After loading the data
df['date'] = pd.to_datetime(df['date'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   title                       10500 non-null  object
 1   link                        10500 non-null  object
 2   date                        10500 non-null  object
 3   source                      10500 non-null  object
 4   number_of_characters_title  10500 non-null  int64 
 5   number_of_words_title       10500 non-null  int64 
 6   day_of_week                 10500 non-null  object
 7   month                       10500 non-null  object
 8   year                        10500 non-null  int64 
 9   quarter                     10500 non-null  int64 
 10  is_weekend                  10500 non-null  bool  
 11  classes_str                 10500 non-null  object
dtypes: bool(1), int64(4), object(7)
memory usage: 912.7+ KB


In [22]:
all_classes = []
for classes_str in df['classes_str']:
    classes = [c.strip() for c in str(classes_str).split(';')]
    all_classes.extend(classes)

# Count occurrences
class_counts = Counter(all_classes)

# Convert to DataFrame
class_df = pd.DataFrame(
    class_counts.items(),
    columns=['class', 'count']
).sort_values('count', ascending=False).reset_index(drop=True)

print(f"\nTotal unique classes found: {len(class_df)}")
print(f"\nTop 12 Most Frequent Classes:\n")
print(class_df.head(15).to_string(index=False))


Total unique classes found: 12

Top 12 Most Frequent Classes:

                                   class  count
                    Work, Jobs & Economy   2526
         Learning, Knowledge & Education   1946
                Technology & Interaction   1733
               Society, Ethics & Culture   1584
           Routine, Lifestyle & Behavior   1479
Sentiment (Positive / Negative Feelings)   1441
                             Human Roles   1256
                   Health, Safety & Risk   1230
       Creativity, Expression & Identity    877
             Cognitive & Decision-Making    832
      Social Interaction & Relationships    730
        Emotion, Motivation & Well-being    551


In [23]:
# DOMAIN_MAP = {
#     "Work": "Work, Jobs & Economy",
#     "Health": "Health, Safety & Risk",
#     "Education": "Learning, Knowledge & Education",
#     "Sentiment": "Sentiment (Positive / Negative Feelings)",
#     "Emotion": "Emotion, Motivation & Well-being",
#     "Human Roles": "Human Roles",
#     "Lifestyle": "Routine, Lifestyle & Behavior:",
#     "Cognitive": "Cognitive & Decision-Making",
#     "Creativity": "Creativity, Expression & Identity",
#     "Relationship": "Social Interaction & Relationships",
#     "Society": "Society, Ethics & Culture",
#     "Technology": "Technology & Interaction"

# }


# Define domain keywords for matching
domain_keywords = {
    'Work': [
        'Work, Jobs & Economy',
        'Work',
        'Jobs',
        'Economy',
        'Human Roles'
    ],
    'Health': [
        'Health, Safety & Risk',
        'Health',
        'Safety',
        'Risk'
    ],
    'Education': [
        'Learning, Knowledge & Education',
        'Learning',
        'Knowledge',
        'Education'
    ],
    'Technology': [
        'Technology & Interaction',
        'Technology',
        'Interaction'
    ],
    'Society': [
        'Society, Ethics & Culture',
        'Society',
        'Ethics',
        'Culture'
    ],
    'Emotion': [
        'Sentiment (Positive / Negative Feelings)',
        'Emotion, Motivation & Well-being',
        'Sentiment',
        'Feelings',
        'Motivation',
        'Well-being'
    ],
    'Creativity': [
        'Creativity, Expression & Identity',
        'Creativity',
        'Expression',
        'Identity'
    ],
    'Behavior': [
        'Routine, Lifestyle & Behavior',
        'Routine',
        'Lifestyle',
        'Behavior'
    ],
    'Cognitive': [
        'Cognitive & Decision-Making',
        'Cognitive',
        'Decision-Making'
    ],
    'Relationships': [
        'Social Interaction & Relationships',
        'Social Interaction',
        'Relationships'
    ]
}

def classify_domain(classes_str, domain_keywords):
    """
    Classify article into domains based on keyword matching.
    An article can belong to multiple domains.
    """
    classes_str_lower = str(classes_str).lower()
    domains = []

    for domain, keywords in domain_keywords.items():
        for keyword in keywords:
            if keyword.lower() in classes_str_lower:
                domains.append(domain)
                break  # Once matched, move to next domain

    return domains if domains else ['Other']

# Apply classification
df['domains'] = df['classes_str'].apply(
    lambda x: classify_domain(x, domain_keywords)
)




In [24]:
# Create boolean columns for key research domains
df['is_work'] = df['domains'].apply(lambda x: 'Work' in x)
df['is_health'] = df['domains'].apply(lambda x: 'Health' in x)
df['is_education'] = df['domains'].apply(lambda x: 'Education' in x)
df['is_emotion'] = df['domains'].apply(lambda x: 'Emotion' in x)
df['is_behavior'] = df['domains'].apply(lambda x: 'Behavior' in x)

# Count articles in each research domain
print("\n Key Research Domain Counts:")
for domain in ['work', 'health', 'education', 'emotion', 'behavior']:
    count = df[f'is_{domain}'].sum()
    percentage = (count / len(df)) * 100
    print(f"  {domain.capitalize():12} {count:5} articles ({percentage:5.1f}%)")


 Key Research Domain Counts:
  Work          3435 articles ( 32.7%)
  Health        1230 articles ( 11.7%)
  Education     1946 articles ( 18.5%)
  Emotion       1711 articles ( 16.3%)
  Behavior      1479 articles ( 14.1%)


In [25]:
# Sample headlines by domain for validation
print("\n📰 Sample Headlines by Key Domains:")
print("="*70)

for domain in ['Work', 'Health', 'Education']:
    filter_col = f'is_{domain.lower()}'
    domain_articles = df[df[filter_col]]

    print(f"\n{domain.upper()} ({len(domain_articles)} articles):")
    for idx, row in domain_articles.head(3).iterrows():
        print(f"  • {row['title']}")


📰 Sample Headlines by Key Domains:

WORK (3435 articles):
  • Google’s AI is the ‘worst’ for stealing content, says People CEO
  • Powering the Next Wave of Enterprise Innovation: Mesh-AI Named OpenAI UK Services Partner
  • AI takes passenger seat in Career Center with Microsoft Copilot

HEALTH (1230 articles):
  • AI chatbot users report mental health issues
  • Can Generative AI transform healthcare?
  • Positioning & Distribution Will Determine the Winning Healthcare AI Startups, Investor Says

EDUCATION (1946 articles):
  • Student Blog: Startups, AI, and Lessons from Singapore
  • West Alabama school district looks to strengthen AI policy
  • CEO: Proposed data center in College Station focused on AI, not Bitcoin mining
